In [1]:
import os
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import mnist

In [2]:
save_dir = "save_dir_8_4"
save_dir_tfrecord = "save_dir_8_4_tfrecord"

# Download data to save_Dir
data_sets = mnist.read_data_sets(save_dir,
                                 dtype=tf.uint8,
                                 reshape=False,
                                 validation_size=1000)
print('{}\n[0]{}\n[1]{}\n[2]{}'.format(type(data_sets),data_sets[0],data_sets[1],data_sets[2]))
print('[0].image.shape:{}\n    labels.shape:{}'.format(data_sets[0].images.shape,data_sets[0].labels.shape))
print('[1].image.shape:{}\n    labels.shape:{}'.format(data_sets[1].images.shape,data_sets[1].labels.shape))
print('[2].image.shape:{}\n    labels.shape:{}'.format(data_sets[2].images.shape,data_sets[2].labels.shape))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting save_dir_8_4/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting save_dir_8_4/train-labels-idx1-ubyte.gz
Extracting save_dir_8_4/t10k-images-idx3-ubyte.gz
Extracting save_dir_8_4/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
<class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'>
[0]<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f1efc558e48>
[1]<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f1efc4f4358>
[2]<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f1efc4f40f0>
[0].image.

In [3]:
# sess = tf.InteractiveSession()

In [4]:
print_onoff = 0
data_splits = ["train","test","validation"]
for d in range(len(data_splits)):
    print('*'*100)
    print("saving "+data_splits[d])
    data_set = data_sets[d]
    filename = os.path.join(save_dir_tfrecord, data_splits[d] + '.tfrecords')
    print('Loop({}) filename : {}'.format(d, filename))
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(data_set.images.shape[0]):
        image = data_set.images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(
            feature={
                'height': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[data_set.images.shape[1]])),     # 실데이터가 아닌, image_row의 height Meta값을 저장한다.
                'width': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[data_set.images.shape[2]])),     # 실데이터가 아닌, image_row의 width Meta값을 저장한다.
                'depth': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[data_set.images.shape[3]])),     # 실데이터가 아닌, image_row의 depth Meta값을 저장한다.
                'label': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[int(data_set.labels[index])])),  # label의 실제값을 저장한다.
                'image_raw': tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[image]))}))                      # image의 실제값을 저장하다.
        print_onoff += 1
        if print_onoff == 1:
            #print("     >> type(example.features.feature) : {}\n     >> example.features.feature['height'] : {}".format(type(example.features.feature),example.features.feature["height"]))
            print("     >> type(example.features.feature['image_row']) : {}".format(type(example.features.feature['image_raw'])))
            print("     >> type(image) : {}   |   len(image) : {}".format(type(image),len(image)))
        writer.write(example.SerializeToString())
    writer.close()
    print('     >> Loop({}) Complete!'.format(d))
print('*'*100)
print('complete!')

****************************************************************************************************
saving train
Loop(0) filename : save_dir_8_4_tfrecord/train.tfrecords
     >> type(example.features.feature['image_row']) : <class 'tensorflow.core.example.feature_pb2.Feature'>
     >> type(image) : <class 'bytes'>   |   len(image) : 784
     >> Loop(0) Complete!
****************************************************************************************************
saving test
Loop(1) filename : save_dir_8_4_tfrecord/test.tfrecords
     >> Loop(1) Complete!
****************************************************************************************************
saving validation
Loop(2) filename : save_dir_8_4_tfrecord/validation.tfrecords
     >> Loop(2) Complete!
****************************************************************************************************
complete!


In [16]:
# READ
NUM_EPOCHS = 10

filename = os.path.join(save_dir_tfrecord, "train.tfrecords")
filename_queue = tf.train.string_input_producer([filename], num_epochs=NUM_EPOCHS)

In [17]:
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized_example,
    features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64),
    })

In [18]:
image = tf.decode_raw(features['image_raw'], tf.uint8)
image.set_shape([784])
image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
label = tf.cast(features['label'], tf.int32)

In [19]:
# Shuffle the examples + batch
images_batch, labels_batch = tf.train.shuffle_batch(
    [image, label], batch_size=128,
    capacity=2000,
    min_after_dequeue=1000)
print('{} {}'.format(images_batch, labels_batch))

Tensor("shuffle_batch_1:0", shape=(128, 784), dtype=float32) Tensor("shuffle_batch_1:1", shape=(128,), dtype=int32)


In [27]:
with tf.variable_scope('W', reuse=tf.AUTO_REUSE):
    W = tf.get_variable("W", [28*28, 10])
 
    y_pred = tf.matmul(images_batch, W)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_pred, labels=labels_batch)
    loss_mean = tf.reduce_mean(loss)

    train_op = tf.train.AdamOptimizer().minimize(loss)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
init = tf.local_variables_initializer()
sess.run(init)

In [28]:
# coordinator
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
for i in range(len(threads)):
    print('({}) : {}'.format(i,threads[i]))

(0) : <Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 139769503807232)>
(1) : <Thread(QueueRunnerThread-input_producer-close_on_stop, started daemon 139769495414528)>
(2) : <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139769463961344)>
(3) : <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-close_on_stop, started daemon 139769927431936)>
(4) : <Thread(QueueRunnerThread-input_producer_1-input_producer_1/input_producer_1_EnqueueMany, started daemon 139770594957056)>
(5) : <Thread(QueueRunnerThread-input_producer_1-close_on_stop, started daemon 139769971472128)>
(6) : <Thread(QueueRunnerThread-shuffle_batch_1/random_shuffle_queue-shuffle_batch_1/random_shuffle_queue_enqueue, started daemon 139769963079424)>
(7) : <Thread(QueueRunnerThread-shuffle_batch_1/random_shuffle_queue-close_on_stop, started daemon 139769950500608)>


In [29]:
try:
    step = 0
    while not coord.should_stop():
        step += 1
        sess.run([train_op])
        if step % 500 == 0:
            loss_mean_val = sess.run([loss_mean])
            print('step:{:10} ,  loss_mean_val : {}'.format(step,loss_mean_val))
except tf.errors.OutOfRangeError:
    print('Done training for %d epochs, %d steps.' % (NUM_EPOCHS, step))
finally:
    # When done, ask the threads to stop.
    coord.request_stop()

coord.join(threads)
sess.close()    

step:       500 ,  loss_mean_val : [0.41213182]
step:      1000 ,  loss_mean_val : [0.3878948]
step:      1500 ,  loss_mean_val : [0.37645757]
step:      2000 ,  loss_mean_val : [0.33515334]
step:      2500 ,  loss_mean_val : [0.34799904]
step:      3000 ,  loss_mean_val : [0.41444105]
step:      3500 ,  loss_mean_val : [0.24963039]
step:      4000 ,  loss_mean_val : [0.41639793]
step:      4500 ,  loss_mean_val : [0.30804473]
Done training for 10 epochs, 4601 steps.
